# CIFAR-10 チュートリアル[転移学習]

## はじめに

以前、CIFAR-10データセットを用いて、CNNを自力で組んで学習させるチュートリアルを書きました。最近（2018/01/18あたり）、転移学習がホットな話題になっているようなので、CIFAR-10を題材に転移学習を扱ってみたいと思います。なお、精度については一切求めていないので注意してください。より高い精度が欲しい場合、ハイパーパラメータを変更する必要があります。


実行環境は以下のとおりです。

In [1]:
import numpy 
numpy.__version__

'1.13.3'

In [2]:
import matplotlib
matplotlib.__version__

'2.0.2'

In [3]:
import tensorflow
tensorflow.__version__

/usr/local/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


'1.4.1'

In [4]:
import keras
keras.__version__

Using TensorFlow backend.


'2.1.2'

## 転移学習

### 転移学習とは

転移学習は英語ではTransfer Learningと言います。なにがTransferなのかは私自身よくわからないのですが、これは何か？大雑把に言えば、「すでに得られている学習結果を、他の学習に使う」手法です。たとえば、すでに大量のカテゴリー分類に対して大量の画像を学習させたデータセット(ImageNetなど)を、もっと小さいデータセットに対して使うことができます。すでに学習させてあるデータセットのネットワークは、（普通）極めて複雑なものですが、この最後の層だけに対して、小さいデータセットを学習させると、より少ないリソース、より少ない学習時間でよい精度の学習結果を得ることができます。この操作を**fine tuning**とか言ったりします。

参考 [転移学習：機械学習の次のフロンティアへの招待](https://qiita.com/icoxfog417/items/48cbf087dd22f1f8c6f4)

この記事では、どういうときに転移学習が使えるのかという可能性の話は脇においておいて、実際問題どういう風に転移学習を行うかに焦点を絞ってみたいと思います。

転移学習、特にfine tuningでいじるのは、ネットワークの構造だけです。というわけで、特に以下の部分に絞って議論していきます。

```
num_classes = 10

# model
model = Sequential()

model.add(Convolution2D(
            32, 
            kernel_size=(3, 3), 
            padding='same',
            input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Convolution2D(
            32, 
            kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(
            (2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(
            64, 
            kernel_size=(3, 3), 
            padding='same'))
model.add(Activation('relu'))
model.add(Convolution2D(
            64, 
            kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))
```

さて、CIFAR-10は画像を10種類に分類するタスクです。転移学習は、**すでに学習させてあるモデルを再利用する**学習なので、どこからかそういうモデル(厳密に言えば.h5ファイル)を取ってこないといけません。また、そのモデルがどういうネットワーク構造で学習されているかも注目する必要があります。というのも、fine tuningでは最後の数層だけを学習させるので、もととなるネットワーク構造が異なれば、結果も異なるからです。

Kerasの[公式ドキュメント](https://keras.io/ja/applications/)を見れば、`Xception`とか`VGG16`とかあると思います。これがネットワーク構造で、実際に私達が組むことができないくらい複雑なものです。ここでは試しにVGG16(というネットワーク構造)を使ってみます。Xceptionも可能ですが、ネットワーク構造が複雑すぎて時間がかかります。

[TensorFlow : 画像分類 : ResNet, Inception-v3, Xception 比較](http://tensorflow.classcat.com/2017/05/15/tensorflow-cnn-model-comparison/)

ところで、VGG16で学習させた結果を使うことはわかりましたが、なにを学習させた結果を使うべきでしょうか？これはImageNetというデータセットで、普通にGPUを使うと1ヶ月以上かかるほど大きなものです。でも安心してください、すでに学習し、保存されているものが使えます。この記事を書いている段階では`14,197,122 images, 21841 synsets indexed`らしいです。半端ないですね。このような大きなデータセットの学習結果を使って、CIFAR-10という小さいカテゴリーの分類タスクを実行してみたいと思います。それがこのチュートリアルの内容です。




### サンプルコード

言葉だけではわかりにくいと思うので、公式ドキュメントから以下のサンプルコードを実行してみたいと思います。すでに学習させているモデルを使うので、内容としては付録Aでやったものと同じです。判別に使う画像は本論で扱った「いらすとや」の車の画像です。初めて実行する際には、大きなモデル(XceptionでImageNetを学習させたモデル)のh5ファイルをダウンロードする必要があります。

In [5]:
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.applications.xception import preprocess_input, decode_predictions
import numpy as np

xce_model = Xception(weights='imagenet')

img_path = 'images/car.png'
img = image.load_img(img_path, target_size=(224, 224))
y = image.img_to_array(img)
y = np.expand_dims(y, axis=0)
y = preprocess_input(y)

preds = xce_model.predict(y)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n02701002', 'ambulance', 0.49092665), ('n04065272', 'recreational_vehicle', 0.45504904), ('n03769881', 'minibus', 0.015279585)]


このように、車を分類することができました。結果は明らかに間違っています。ただここでの問題は、私達が扱いたいのはたったの10種類の分類タスクなので、ambulanceかどうかは関係ありません。そこまで細かいカテゴリー分類は求めていないんですね。そのため、以降「CIFAR-10で扱う10種類のカテゴリーに正しく分類できれば良い」とします。

## 転移学習を行う - Fine Tuning -

では、実際にfine tuningを行って、CIFAR-10に対して転移学習を適用させてみます。

データセットの準備までは同じなので、以下を実行します。

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import dlipr
import os

from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.xception import preprocess_input, decode_predictions
from keras.callbacks import EarlyStopping

# ---------------------------------------------------------
# Load and preprocess data
# ---------------------------------------------------------
data = dlipr.cifar.load_cifar10()

# preprocess the data in a suitable way
# reshape the image matrices to vectors
#RGB 255 = white, 0 = black
X_train = data.train_images.reshape([-1, 32, 32, 3])
X_test = data.test_images.reshape([-1, 32, 32, 3])
print('%i training samples' % X_train.shape[0])
print('%i test samples' % X_test.shape[0])
print(X_train.shape)

# convert integer RGB values (0-255) to float values (0-1)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# convert class labels to one-hot encodings
Y_train = to_categorical(data.train_labels, 10)
Y_test = to_categorical(data.test_labels, 10)

50000 training samples
10000 test samples
(50000, 32, 32, 3)


次に層を重ねていきますが、Xceptionの構造を使うので、すべての層を通過した後のモデルのインスタンスを`base_model`として取り出します。

In [7]:
# create the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False)

その後で、Xceptionのネットワーク構造を固定します。

In [8]:
for layer in base_model.layers:
    layer.trainable = False

ただし、`base_model`のあとで追加された層に関しては、CIFAR-10の学習で影響を受けます。すなわち、新たにウエイトが決まることになります。

In [9]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(10, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

ネットワークの構造が決まれば後は同様です。今回は`Earlystopping`を`callback`に追加しておくことにします。

In [10]:
print(model.summary())

model.compile(
loss='categorical_crossentropy',
optimizer=Adam(lr=0.001),
metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

fit = model.fit(X_train, Y_train,
              batch_size=128,
              epochs=40, #shouldn't be raised to 100, because the overfitting occurs.
              verbose=2,
              validation_split=0.1,
              callbacks=[es]
                )

score = model.evaluate(X_test, Y_test,
                    verbose=0
                    )
print('Test score:', score[0])
print('Test accuracy:', score[1])

# ----------------------------------------------
# Some plots
# ----------------------------------------------

# make output directory
folder = 'results'
if not os.path.exists(folder):
    os.makedirs(folder)
    
model.save(os.path.join(folder, 'my_model-tl.h5'))

# predicted probabilities for the test set
Yp = model.predict(X_test)
yp = np.argmax(Yp, axis=1)

# plot some test images along with the prediction
for i in range(10):
    dlipr.utils.plot_prediction(
        Yp[i],
        data.test_images[i],
        data.test_labels[i],
        data.classes,
        fname=os.path.join(folder, 'test-%i.png' % i))

fig, (axL, axR) = plt.subplots(ncols=2, figsize=(10,4))

# loss
def plot_history_loss(fit):
    # Plot the loss in the history
    axL.plot(fit.history['loss'],label="loss for training")
    axL.plot(fit.history['val_loss'],label="loss for validation")
    axL.set_title('model loss')
    axL.set_xlabel('epoch')
    axL.set_ylabel('loss')
    axL.legend(loc='upper right')

# acc
def plot_history_acc(fit):
    # Plot the loss in the history
    axR.plot(fit.history['acc'],label="loss for training")
    axR.plot(fit.history['val_acc'],label="loss for validation")
    axR.set_title('model accuracy')
    axR.set_xlabel('epoch')
    axR.set_ylabel('accuracy')
    axR.legend(loc='upper right')

plot_history_loss(fit)
plot_history_acc(fit)
fig.savefig(os.path.join(folder, 'cifar10-tutorial-tl.png'))
plt.close()

Train on 45000 samples, validate on 5000 samples
Epoch 1/40
 - 1395s - loss: 1.3682 - acc: 0.5227 - val_loss: 1.2142 - val_acc: 0.5686
Epoch 2/40
 - 1593s - loss: 1.1706 - acc: 0.5902 - val_loss: 1.1308 - val_acc: 0.6032
Epoch 3/40
 - 1717s - loss: 1.1004 - acc: 0.6152 - val_loss: 1.1175 - val_acc: 0.6118
Epoch 4/40
 - 1714s - loss: 1.0435 - acc: 0.6345 - val_loss: 1.0917 - val_acc: 0.6164
Epoch 5/40
 - 1715s - loss: 0.9950 - acc: 0.6526 - val_loss: 1.0746 - val_acc: 0.6180
Epoch 6/40
 - 1611s - loss: 0.9492 - acc: 0.6672 - val_loss: 1.0611 - val_acc: 0.6260
Epoch 7/40
 - 873s - loss: 0.9130 - acc: 0.6799 - val_loss: 1.0737 - val_acc: 0.6268
Epoch 8/40
 - 875s - loss: 0.8717 - acc: 0.6941 - val_loss: 1.0665 - val_acc: 0.6296
Epoch 9/40
 - 772s - loss: 0.8308 - acc: 0.7112 - val_loss: 1.0576 - val_acc: 0.6374
Epoch 10/40
 - 930s - loss: 0.7930 - acc: 0.7241 - val_loss: 1.0613 - val_acc: 0.6318
Epoch 11/40
 - 823s - loss: 0.7539 - acc: 0.7378 - val_loss: 1.0596 - val_acc: 0.6354
Epoch 12

<img src='results/cifar10-tutorial-tl.png'/>

となり、精度62.77%が得られました。結果は本論で組んだネットワークのものより悪くなりましたが、これはネットワーク構造やハイパーパラメータのためです。ネットワークの構造が変わっているので、fine tuningの段階でハイパーパラメータを調整しないとよい精度は得られません。

転移学習については以上のような段階を踏みます。VGG16内部のネットワーク構造の最後の数枚の層をfreezeさせたりすることもできますが、それはサンプルコード等から試されることをおすすめします(~~リソースが足りない~~)。